In [5]:
import sys
sys.path.append("..")
from lib import get_max_day8 as get_max
import numpy as np
import re

Test input:
- With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Input:
- Consider your map; how many trees are visible from outside the grid?



In [16]:
class Forest:
    def __init__(self, filename : str, verbose : bool = False):
        with open(filename) as file:
            input = file.read()
        lines = re.split("\n", input)
        self.n_rows = len(lines)
        self.n_cols = len(lines[0])
        print(f"Contains {self.n_rows} rows / {self.n_cols} columns.") if verbose else None
        del lines
        input = re.sub("\s","", input)
        input = [int(letter) for letter in input]
        self.trees = np.array(input).reshape((self.n_rows, self.n_cols))
        print(f"{self.trees}") if verbose else None
        return
    
    def check_direction(self, row : int, col : int, dir : str) -> bool:
        # check for direction, if all trees are lower than at (row,col)
        height = self.trees[row, col]
        if dir == 'L': # Left
            result = get_max(self.trees[row,:col]) < height
        elif dir == 'R': # Right
            result = get_max(self.trees[row,col+1:]) < height
        elif dir == 'U': # Up
            result = get_max(self.trees[:row,col]) < height
        elif dir == 'D': # Down
            result = get_max(self.trees[row+1:,col]) < height
        else:
            NotImplementedError
        return result
    
    def compute_scenic_score(self, row : int, col : int) -> int:
        def count_dist(value : int, scope) -> int:
            count = 0
            for element in scope:
                count += 1
                if element >= value:
                    break
            return count
        score = 1
        value = self.trees[row,col]
        scope = self.trees[row,:col] # to the left
        score *= count_dist(value, reversed(scope))
        scope = self.trees[row,col+1:] # to the right
        score *= count_dist(value, scope)
        scope = self.trees[:row,col] # upwards
        score *= count_dist(value, reversed(scope))
        scope = self.trees[row+1:,col] # downwards
        score *= count_dist(value, scope)
        return score

    def get_best_scenic_score(self) -> int:
        tracker = 0
        for i in range(self.n_rows):
            for j in range(self.n_cols):
                res = self.compute_scenic_score(i,j)
                if res >= tracker:
                    tracker = res
        return tracker

    def is_visible(self, row : int, col : int, verbose : bool = False) -> bool:
        result = self.check_direction(row, col, 'L') or self.check_direction(row, col, 'R') or self.check_direction(row, col, 'U') or self.check_direction(row, col, 'D')
        print(f"Row {row} Col {col} is visible") if result and verbose else None
        print(f"Row {row} Col {col} is not visible") if (not result) and verbose else None
        return result

    def count_visible(self, verbose : bool = False) -> int:
        count = 0
        for i in range(self.n_rows):
            for j in range(self.n_cols):
                count += int(self.is_visible(i, j, verbose))
        return count

In [19]:
def test_checkDirection():
    forest = Forest("test_input.txt", True)
    print(f"Soll: True / Ist: {forest.check_direction(0,0,'U')}")
    print(f"Soll: False / Ist: {forest.check_direction(1,1,'D')}")
    print(f"Soll: True / Ist: {forest.check_direction(4,3,'L')}")
    return

def test_checker():
    forest = Forest("test_input.txt", True)
    n_visible = forest.count_visible()
    print(f"Test input has {n_visible} trees.")
    print("Test A successful") if n_visible == 21 else print("Test A not successful")
    n_best_scenic_score = forest.get_best_scenic_score()
    print(f"Test input has best scenic score {n_best_scenic_score}")
    print("Test B successful") if n_best_scenic_score == 8 else print("Test B not successful")
    return

def compute_result(filename : str):
    forest = Forest(filename)
    n_visible = forest.count_visible()
    print(f"Input has {n_visible} visible trees.")
    n_best_scenic_score = forest.get_best_scenic_score()
    print(f"Input has {n_best_scenic_score} as best scenic score.")
    return    

print("Testing part a...")
test_checkDirection()
test_checker()

print("Getting answer for part a and b now...")
compute_result("input.txt")

Testing part a...
Contains 5 rows / 5 columns.
[[3 0 3 7 3]
 [2 5 5 1 2]
 [6 5 3 3 2]
 [3 3 5 4 9]
 [3 5 3 9 0]]
Soll: True / Ist: True
Soll: False / Ist: False
Soll: True / Ist: True
Contains 5 rows / 5 columns.
[[3 0 3 7 3]
 [2 5 5 1 2]
 [6 5 3 3 2]
 [3 3 5 4 9]
 [3 5 3 9 0]]
Test input has 21 trees.
Test A successful
Test input has best scenic score 8
Test B successful
Getting answer for part a and b now...
Input has 1684 visible trees.
Input has 486540 as best scenic score.
